In [1]:
import gzip
import json
import pandas as pd
import re
import enum
from sqlalchemy import MetaData, Table, Column, Integer, String, Boolean, Text, Float, TIMESTAMP, Enum
from db_utils import *


## 1. Review Existing Unstructured Data and Diagram a New Structured Relational Data Model


### 1a. Read json data into DataFrame Objects

In [2]:
jsonFiles = ["brands.json.gz", "receipts.json.gz", "users.json.gz"]

def readJsonGz(file_path):
    """
    Given a path to a json.gz file, `file_path`
    Read the data into a normalized pandas DataFrame object

    Args:
        file_path(str): The path to a gzip file

    Returns:
        a pandas DataFrame object containing data in `file_path`
    """

    with gzip.open(file_path, 'r') as data_file:    
        # data = json.load(data_file)  
        data = data_file.read()

    data = data.decode("utf-8")
    lines = data.split('\n')

    for i, line in enumerate(lines):
        if line.strip():  # Skip empty lines
            try:
                json.loads(line)
            except json.JSONDecodeError as e:
                print(f"Invalid JSON on line {i+1}: {e}")
                print(f"Content: {line}")
                break

    data = "[" + ",".join(line for line in lines if line.strip()) + "]"
    data = json.loads(data)
    normalized_data = pd.json_normalize(data)
    return normalized_data

def extractData(line):
    """
    Use regex to extract data wrapped between '{' and '}' in an input string.
    Args: 
        line(str): The data stored in a line of json file

    Returns: 
        substring of `line`, contains characters wrapped around by {}
    """
    pattern = r'\{.*\}'

    # Search for the pattern in the input string
    match = re.search(pattern, line)
    json_object =""

    if match:
        json_object = match.group(0)
    return json_object


def read_json_gz_skip_gzip_lines(file_path):
    """
    Given a path to a json.gz file, `file_path`
    Read the data into a json object and return

    Args:
        file_path(str): The path to a gzip file
        
    Returns:
        a json object containing data in `file_path`
    """
    valid_lines = []

    # Open the gzip file
    with gzip.open(file_path, 'r') as f:
        d = f.read()
        lines = d.decode("utf-8").split("\n")
        valid_lines = []
        for line in lines:
            extracted = extractData(line)
            if extracted != "":
                valid_lines.append(extracted)
    json_string =  "[" + ",".join(valid_lines) + "]"
    data = json.loads(json_string)
    
    return data

rewardsReceiptItemsLst = []
num_nan = 0


def normalize_rewardsReceiptItems(row, dest = rewardsReceiptItemsLst):
    """
    Extract 'reward receipt item' from column `rewardsReceiptItemList`
    and map receipt id to each item.

    Args:
        row(Pandas series): a pandas series sliced from a dataframe as a row
        dest(list): a list where each processed item will be appended to
    
    Returns:
        None
    """
    global num_nan
    receipt_id = row["_id"]
    rewardsReceiptItemList = row["rewardsReceiptItemList"]
    if pd.isna(rewardsReceiptItemList).all() if isinstance(rewardsReceiptItemList, (list, pd.Series)) else pd.isna(rewardsReceiptItemList):
        num_nan += 1
        return 
    new_row = pd.json_normalize(rewardsReceiptItemList)
    new_row["receiptId"] = receipt_id
    new_row["dateScanned"] = row["dateScanned"]
    new_row["userId"] = row["userId"]

    dest.append(new_row)



Found that users.json.gz is not able to be imported due to unexpected symbols.  
Unzip it, and try to investigate

In [3]:
dataDict = {}
for file in jsonFiles:
    try:
        data= readJsonGz(file)
        dataDict[file.split(".")[0]] = data
    except json.JSONDecodeError as e:
        print(e)
        print("Fail to read {}\n".format(file))


Invalid JSON on line 1: Expecting value: line 1 column 1 (char 0)
Content: users.json                                                                                          0100777 0000000 0000000 00000254301 14567170355 010072  0                                                                                                    ustar 00                                                                                                                                                                                                                                                       {"_id":{"$oid":"5ff1e194b6a9d73a3a9f1052"},"active":true,"createdDate":{"$date":1609687444800},"lastLogin":{"$date":1609687537858},"role":"consumer","signUpSource":"Email","state":"WI"}
Expecting value: line 1 column 2 (char 1)
Fail to read users.json.gz



In [4]:
# !gzip -dk users.json.gz

Opened the users.json file and found that  
1)  in the first line,
there're bunch of "\x00" characters in front of a actual data row.  
2) In the last row, it's full of "\x00".  
Therefore, created a function `read_json_gz_skip_gzip_lines` to extract data placed between "{}" in each line

In [5]:
file = jsonFiles[2]
try:
    data= read_json_gz_skip_gzip_lines(file)
    dataDict[file.split(".")[0]] = pd.json_normalize(data)
except json.JSONDecodeError:
    print(e)
    print(file)
dataDict.keys()

dict_keys(['brands', 'receipts', 'users'])

In [6]:
dataDict.keys()

dict_keys(['brands', 'receipts', 'users'])

In [7]:
def cleanColName(col):
    splitted = col.split(".")
    if len(splitted) <= 2:
        return splitted[0]
    else:
        return col.replace("$", "").replace(".", "_")

In [8]:
# Load raw processed data from dataDict
brandsData = dataDict["brands"]
receiptsData = dataDict["receipts"]
usersData = dataDict["users"]

# Clean user table columns
usersData.role = usersData.role.apply(str.upper)
usersDataRenameCols = {col: cleanColName(col) for col in usersData.columns if "." in col}
usersData = usersData.rename(columns=usersDataRenameCols)

# Clean brand table columns
brandsDataRenameCols = {'_id.$oid': '_id',
                        'cpg.$id.$oid': 'cpg_id_oid', 'cpg.$ref': 'cpg_ref'}
brandsData = brandsData.rename(columns=brandsDataRenameCols)

# Clean receipt table columns
receiptsDataRenameCols = {col: cleanColName(col) for col in receiptsData.columns if "." in col}
receiptsData = receiptsData.rename(columns=receiptsDataRenameCols)

# Convert all date columns to date object
for col in ['createdDate','lastLogin']:
    usersData[col] = pd.to_datetime(usersData[col], unit="ms")

# Convert all date columns to date object
for col in ['createDate', 'dateScanned',
        'finishedDate', 'modifyDate', 
        'pointsAwardedDate', 'purchaseDate']:
    receiptsData[col] = pd.to_datetime(receiptsData[col], unit="ms")

Found that Receipt dataframe has a column that contains list of other objects: *rewardsReceiptItemList*

In [9]:
receiptsData.head()

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,purchaseDate
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,2021-01-03 00:00:00
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,2021-01-02 15:24:43
2,5.0,All-receipts receipt bonus,5,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,2021-01-03 00:00:00
3,5.0,All-receipts receipt bonus,5.0,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,2021-01-03 00:00:00
4,5.0,All-receipts receipt bonus,5.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,2021-01-02 15:25:06


In [10]:
# Receipts table has a column that contains list of other objects
receiptsData.apply(normalize_rewardsReceiptItems, axis = 1)
rewardsReceiptItems = pd.concat(rewardsReceiptItemsLst, ignore_index= True)
rewardsReceiptItems.head()

,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,userFlaggedNewItem,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Validate data integrity after applying data extraction to receipt table

In [11]:
actual_nan_rri = receiptsData["_id"].count() - receiptsData.rewardsReceiptItemList.count()

report1 = """
During extracting receipt reward items, found {} receipts that has missing value in column 'receiptsRewordsItemList'\n
Matched with actual missing values in the column: {}\n
Total number of receipt rewards items extracted: {}\n
"""
print(report1.format(num_nan, actual_nan_rri == num_nan, rewardsReceiptItems.shape[0] ))



During extracting receipt reward items, found 440 receipts that has missing value in column 'receiptsRewordsItemList'

Matched with actual missing values in the column: True

Total number of receipt rewards items extracted: 6941




In [12]:
receiptsData.columns

Index(['bonusPointsEarned', 'bonusPointsEarnedReason', 'pointsEarned',
       'purchasedItemCount', 'rewardsReceiptItemList', 'rewardsReceiptStatus',
       'totalSpent', 'userId', '_id', 'createDate', 'dateScanned',
       'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate'],
      dtype='object')

In [13]:
# Since rewardsReceiptItemList has been extracted and saved to a table,
# drop this column from receipt table
receiptsCols = ['bonusPointsEarned', 'bonusPointsEarnedReason', 'pointsEarned',
                'purchasedItemCount', 'rewardsReceiptStatus',
                'totalSpent', 'userId', '_id', 'createDate',
                'dateScanned', 'finishedDate', 'modifyDate',
                'pointsAwardedDate', 'purchaseDate']
receiptsData = receiptsData[receiptsCols]

In [14]:
# Connect to database

db = 'fetch_db'
host = 'database-1.crgewaiyut90.us-east-2.rds.amazonaws.com'
user = 'admin'
pw = 'wP7Fkg4jn7kEj'

conn = create_connection(host, user, pw, db)
engine = create_sqlalchemy_engine(user, pw, host, db)


Connection to MySQL DB successful


Create database in the MySQL server

In [15]:
# create_db_query = f"""CREATE DATABASE {db}"""
use_db_query = f"""USE {db}"""

execute_query(conn, use_db_query)

Query executed successfully


Load tables into Database

a) Load user table into Database

In [16]:
print(list(receiptsData.rewardsReceiptStatus.unique()))
print(list(usersData.role.unique()))


['FINISHED', 'REJECTED', 'FLAGGED', 'SUBMITTED', 'PENDING']
['CONSUMER', 'FETCH-STAFF']


In [17]:
class RewardsStatus(enum.Enum):
    FINISHED = "FINISHED"
    SUBMITTED = "SUBMITTED"
    REJECTED = "REJECTED"
    PENDING = "PENDING"
    FLAGGED = "FLAGGED"

class UserRole(enum.Enum):
    CONSUMER = "CONSUMER"
    FETCH_STAFF = "FETCH_STAFF"

USER_TABLE = "User"
RECEIPT_TABLE = "Receipt"
BRAND_TABLE = "Brand"
ITEM_TABLE = "ReceiptRewardItem"



metadata = MetaData()

# Define the table schema
user_table = Table(
    USER_TABLE, metadata,
    Column('active', Boolean),
    Column('role', Enum(UserRole, name="UserRole")),
    Column('signUpSource', Text),
    Column('state', Text),
    Column('_id',  String(255), primary_key=True),
    Column('createdDate', TIMESTAMP),
    Column('lastLogin', TIMESTAMP)
)


# Define the table schema
receipt_table = Table(
    RECEIPT_TABLE, metadata,
    Column('_id', String(255), primary_key=True),
    Column('bonusPointsEarned', Integer),
    Column('bonusPointsEarnedReason', Text),
    Column('pointsEarned', Integer),
    Column('purchasedItemCount', Integer),
    Column('totalSpent', Float),
    Column('userId', String(255)),
    Column('rewardsReceiptStatus', Enum(RewardsStatus, name="RewardsStatus")),
    Column('createDate', TIMESTAMP),
    Column('dateScanned', TIMESTAMP),
    Column('finishedDate', TIMESTAMP),
    Column('modifyDate', TIMESTAMP),
    Column('pointsAwardedDate', TIMESTAMP),
    Column('purchaseDate', TIMESTAMP)
)

brand_table = Table(
    BRAND_TABLE, metadata,
    Column('_id',  String(255), primary_key=True),
    Column('barcode', String(255)),
    Column('category', String(255)),
    Column('categoryCode', String(255)),
    Column('name', String(255)),
    Column('topBrand', Boolean),
    Column('cpg_id_oid', String(255)),
    Column('cpg_ref', String(255)),
    Column('brandCode', String(255))
)

receipt_reward_item = Table(
    ITEM_TABLE, metadata,
    Column('barcode', String(255)),
    Column('brandCode', String(255)),
    Column('receiptId', String(255)),
    Column('userId', String(255)),
    Column('dateScanned', TIMESTAMP),
    Column('partnerItemId', String(255)),
    Column('pointsPayerId', String(255)),
    Column('rewardsProductPartnerId', String(255)),
    Column('metabriteCampaignId', String(255)),
    Column('description', String(255)),
    Column('finalPrice', Float),
    Column('itemPrice', Float),
    Column('needsFetchReview', Boolean),
    Column('preventTargetGapPoints', Boolean),
    Column('quantityPurchased', Integer),
    Column('userFlaggedBarcode', String(255)),
    Column('userFlaggedNewItem', Boolean),
    Column('userFlaggedPrice', Float),
    Column('userFlaggedQuantity', Integer),
    Column('needsFetchReviewReason', String(255)),
    Column('pointsNotAwardedReason', String(255)),
    Column('rewardsGroup', String(255)),
    Column('userFlaggedDescription', String(255)),
    Column('originalMetaBriteBarcode', String(255)),
    Column('originalMetaBriteDescription', String(255)),
    Column('competitorRewardsGroup', String(255)),
    Column('discountedItemPrice', Float),
    Column('originalReceiptItemText', String(255)),
    Column('itemNumber', String(255)),
    Column('originalMetaBriteQuantityPurchased', Integer),
    Column('pointsEarned', Float),
    Column('targetPrice', Float),
    Column('competitiveProduct', Boolean),
    Column('originalFinalPrice', Float),
    Column('originalMetaBriteItemPrice', Float),
    Column('deleted', Boolean),
    Column('priceAfterCoupon', Float)
)

metadata.create_all(engine)

In [18]:
# Add foreign key constraints, during this process, 
# I found that not all use ids referenced in Receipt table exist in the User table, therefore, here can't apply this constraint
# set_constraint_receipt_query = f"""
#         ALTER TABLE {RECEIPT_TABLE}
#         ADD FOREIGN KEY (userId) REFERENCES User(_id);"""

set_constraint_item_query = f"""
        ALTER TABLE {ITEM_TABLE}
        ADD FOREIGN KEY (receiptId) REFERENCES Receipt(_id);"""

# set_constraint_item_query2 = f"""
#         ALTER TABLE {ITEM_TABLE}
#         ADD FOREIGN KEY (userId) REFERENCES User(_id);"""

In [19]:
# execute_query(conn, set_constraint_receipt_query)
execute_query(conn, set_constraint_item_query)
# execute_query(conn, set_constraint_item_query2)


Query executed successfully


In [20]:
usersData.role.value_counts()

role
CONSUMER       413
FETCH-STAFF     82
Name: count, dtype: int64

In [21]:
usersData._id.value_counts()

_id
54943462e4b07e684157a532    20
5fc961c3b8cfca11a077dd33    20
5ff5d15aeb7c7d12096d91a2    18
5fa41775898c7a11a6bcef3e    18
59c124bae4b0299e55b0f330    18
                            ..
6004a965e257124ec6b9a39f     1
600746fd6e64691717e8cfb5     1
60074b996e64691717e8f11a     1
60074246325c8a12289e22a0     1
60088e5d633aab121bb8e5cf     1
Name: count, Length: 212, dtype: int64

From above, found that there are duplicates in the User table.

In [22]:
d = {'CONSUMER':'CONSUMER',
     'FETCH-STAFF': 'FETCH_STAFF'} # SQL converts "-" to "_"
usersData.role = usersData.role.apply(lambda x: d[x])

In [23]:
usersData.drop_duplicates(inplace= True) #default use all columns to determine duplicate
# Load data into database table `user`
usersData.to_sql(name = "User", con=engine, if_exists='append', index=False)

212

In [24]:
hset = set(usersData._id)
unknownUserIds = []
for idx, id in enumerate(receiptsData.userId):
    if id not in hset:
        unknownUserIds.append(id)
print("{} user ids in Receipt table that doesn't have matched user ids in User tables\n".format(len(unknownUserIds)))

148 user ids in Receipt table that doesn't have matched user ids in User tables



In [25]:
# b) Load receipt table into database
print("number of receipt id that has duplicate {}\n".format(sum(receiptsData._id.value_counts() > 1)))
receiptsData.to_sql(name = "Receipt", con=engine, if_exists='append', index=False)

# c) Load Brand table into database
brandsData.to_sql(name = "Brand", con=engine, if_exists='append', index=False)

# d) Load ReceiptRewardItem into database
rewardsReceiptItems.to_sql(name = "ReceiptRewardItem", con=engine, if_exists='append', index=False)

number of receipt id that has duplicate 0



6941

## 2. Write queries that directly answer predetermined questions from a business stakeholder

#### 2a. What are the top 5 brands by receipts scanned for most recent month?

1. The most recent month in the dataset is "2021-03". However, none of the receipts scanned in this month have a brandCode in the receipt record. Unless I missed out that there is another column that could relate to the Brand data in the receipt table, we have to push back months to the most recent month that has receipts of non-missing brandCode to get the top five brands scanned for most recent month.  

2. Use `query1`, it finds the most recent month that has receipts which has valid brandCode, and count the total number of receipts scanned that purchased items from each brand, then get the top 5 brands that has the highest number of receipts scanned.  
The result shows that "2021-02" is the most recent month that has receipts with valid brandCode, and there are in total 3 such brands.

In [26]:
query_to_df(engine, """
            SELECT DATE_FORMAT(MAX(dateScanned), '%Y-%m') AS most_recent_month 
            FROM ReceiptRewardItem
            """)

,most_recent_month
0,2021-03


In [27]:
# What are the top 5 brands by receipts scanned for most recent month?
query1 = """
    WITH 
        MostRecentMonth AS (
            SELECT DATE_FORMAT(MAX(dateScanned), '%Y-%m') AS most_recent_month 
                    FROM ReceiptRewardItem  
            WHERE ReceiptRewardItem.brandCode <> 'NULL'
        )

    SELECT COUNT(DISTINCT(receiptId)) as numReceipts,
            brandCode,
            (SELECT most_recent_month FROM MostRecentMonth) AS most_recent_month
    FROM ReceiptRewardItem
    WHERE brandCode <> 'NULL' AND 
        DATE_FORMAT(dateScanned, '%Y-%m') = (SELECT most_recent_month FROM MostRecentMonth)
    GROUP BY brandCode
    ORDER BY numReceipts DESC
    LIMIT 5
"""

In [28]:
df = query_to_df(engine, query1)
# df.sort_values(by= ["dateScanned"], ascending=False)
df

,numReceipts,brandCode,most_recent_month
0,3,BRAND,2021-02
1,2,MISSION,2021-02
2,1,VIVA,2021-02


#### 2b. How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

Adopt a similar strategy, in the query below, it first find the second most recent month such that has receipts with brandCode. Then use it to filter out all receipts scanned in that month.
The results show that the five brands that has the most receipts scanned are:  
'BEN AND JERRYS', 'PEPSI', 'FOLGERS', 'KRAFT', "KELLOGG'S"

Compared the the result from executing query1, "BRAND", "MISSION", "VIVA", they are no overlapps. 
The 5 brands has the most recetips scanned in the previous recent month are not in the list of the top 5 in the most recent month at all.

In [29]:
query2 = """
    WITH 
        SecondMostRecentMonth AS (
            SELECT DISTINCT DATE_FORMAT(dateScanned, '%Y-%m') AS month 
                    FROM ReceiptRewardItem  
            WHERE ReceiptRewardItem.brandCode <> 'NULL'
            ORDER BY month DESC
            LIMIT 1 OFFSET 1
        )

    SELECT COUNT(DISTINCT(receiptId)) as numReceipts,
            brandCode as Top5,
            (SELECT month FROM SecondMostRecentMonth) AS second_most_recent_month
    FROM ReceiptRewardItem
    WHERE brandCode <> 'NULL' AND 
            DATE_FORMAT(dateScanned, '%Y-%m') = (SELECT month FROM SecondMostRecentMonth)
    GROUP BY brandCode
    ORDER BY numReceipts DESC
    LIMIT 5
"""
res2 = query_to_df(engine, query2)
res2

,numReceipts,Top5,second_most_recent_month
0,32,BEN AND JERRYS,2021-01
1,23,PEPSI,2021-01
2,23,FOLGERS,2021-01
3,22,KRAFT,2021-01
4,22,KELLOGG'S,2021-01


#### 2c. When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

Findings:   
1. According to the values of rewardsReceiptStatus’:  
    "FINISHED",  
    "SUBMITTED",  
    "REJECTED",   
    "PENDING",   
    "FLAGGED",   
I can safely assume that "FINISHED" is equivalent to status "Accepted" 
2. According to the result of the following query, receipts with "Accecpted" reward status has higher average spend

In [30]:
query3 = """
    SELECT AVG(totalSpent), 
        CASE 
            WHEN rewardsReceiptStatus = 'FINISHED' THEN 'ACCEPTED'
            ELSE rewardsReceiptStatus 
        END AS rewardsReceiptStatus 
    FROM Receipt
    WHERE totalSpent <> 'NULL'
    GROUP BY rewardsReceiptStatus
    HAVING rewardsReceiptStatus = 'FINISHED' OR rewardsReceiptStatus = 'REJECTED'
        """
query_to_df(conn=engine, query = query3)

,AVG(totalSpent),rewardsReceiptStatus
0,24.355146,REJECTED
1,81.167694,ACCEPTED


#### 2d. When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?



Findings: 
1. 1. According to the values of rewardsReceiptStatus’:  
    "FINISHED",  
    "SUBMITTED",  
    "REJECTED",   
    "PENDING",  
    "FLAGGED",   
I can safely assume that "FINISHED" is equivalent to status "Accepted"   
2. ##### According to the result of the following query, receipts with "Accecpted" reward status has higher total number of items purchaed

In [31]:
query4 = """
    SELECT SUM(purchasedItemCount) as Total, 
            CASE 
                WHEN rewardsReceiptStatus = 'FINISHED' THEN 'ACCEPTED'
                ELSE rewardsReceiptStatus 
            END AS rewardsReceiptStatus  
    FROM Receipt
    WHERE purchasedItemCount <> 'NULL'
    GROUP BY rewardsReceiptStatus
    HAVING rewardsReceiptStatus = 'FINISHED' OR rewardsReceiptStatus = 'REJECTED'
    """
query_to_df(conn=engine, query = query4)

,Total,rewardsReceiptStatus
0,173.0,REJECTED
1,8184.0,ACCEPTED


#### 2e. Which brand has the most spend among users who were created within the past 6 months?
Assuming "within the past 6 months" include current month. 

Considerations:  
i. if consider the current month is same as the month of today, the query I would use is:   
"""  
    WITH  
        receiptsOfRecentUsers AS (  
        Select r._id, r.totalSpent, u.createdDate  
        FROM receipt AS r LEFT JOIN user as u ON r.userId = u._id  
        WHERE DATE_FORMAT(u.createdDate, '%Y-%m') >= DATE_FORMAT(DATE_SUB(curdate(), INTERVAL 5 MONTH), '%Y-%m')  
        )  
    SELECT item.brandCode, ROUND(SUM(r.totalSpent), 2) AS Total  
    FROM receiptsOfRecentUsers AS r  
    LEFT JOIN ReceiptRewardItem AS item ON r._id = item.receiptId
    WHERE item.brandCode <> 'NULL'
    GROUP BY item.brandCode
    ORDER BY Total DESC
    LIMIT 1;  
"""  
   
ii. However, I found that the data given is up tp "2021-03" based on both the createdDate in User table, and the scannedDate in Receipt table.  

Therefore, based on the given data:  
How do I define "past 6 month": assuming the most recent month in the dataset is the most recent moneth, which is "2021-03". 
The most recent month by dateScanned in receipt table is "2021-03", them most recent month by createdDate in user table is "2021-02", so I pick the maximum among the two as the most recent month.

Then I trace backward 5 months and include the most recent month, therefore the past 6 months are from "2020-10" to "2021-03".  

ii. Filter out all receipts that were scanned by users created within the past 6 months 
 
iii. Join these receipts with items, group by brand, and sum up the total spent


##### Finding(s): 'BEN AND JERRYS' is the brand has the most total spend among users who were created within the past 6 months.


In [32]:
query5 = """
    WITH maxMonth AS (
        SELECT MAX(month) AS max_month
        FROM (
            (SELECT dateScanned AS month
            FROM Receipt
            ORDER BY dateScanned DESC
            LIMIT 1)
            UNION ALL
            (SELECT createdDate AS month
            FROM User
            ORDER BY createdDate DESC
            LIMIT 1)
            ) AS combined_results
        ),
        receiptsOfRecentUsers AS (
            SELECT r._id, r.totalSpent, u.createdDate
            FROM Receipt AS r
            LEFT JOIN User AS u ON r.userId = u._id
            WHERE DATE_FORMAT(u.createdDate, '%Y-%m') >= (SELECT DATE_FORMAT(DATE_SUB(max_month, INTERVAL 5 MONTH), '%Y-%m') FROM maxMonth)
        )
    SELECT item.brandCode, ROUND(SUM(r.totalSpent), 2) AS Total
    FROM receiptsOfRecentUsers AS r
    LEFT JOIN ReceiptRewardItem AS item ON r._id = item.receiptId
    WHERE item.brandCode <> 'NULL'
    GROUP BY item.brandCode
    ORDER BY Total DESC
    LIMIT 1;
    """
query_to_df(conn=engine, query = query5)

,brandCode,Total
0,BEN AND JERRYS,197337.68


#### 2f. Which brand has the most transactions among users who were created within the past 6 months?  
Assuming "within the past 6 months" include current month  

Considerations
i. if consider the current month is same as the month of today, the query I would use is:   
"""  
    WITH  
        receiptsOfRecentUsers AS (  
        Select r._id, r.totalSpent, u.createdDate  
        FROM receipt AS r LEFT JOIN user as u ON r.userId = u._id  
        WHERE DATE_FORMAT(u.createdDate, '%Y-%m') >= DATE_FORMAT(DATE_SUB(curdate(), INTERVAL 5 MONTH), '%Y-%m')  
        )  
    SELECT item.brandCode, COUNT(*) as Total  
        FROM receiptsOfRecentUsers AS r LEFT JOIN ReceiptRewardItem AS item  
            ON r._id = item.receiptId  
        WHERE item.brandCode <> 'NULL'  
        GROUP BY item.brandCode  
        ORDER BY Total DESC  
        LIMIT 1  
"""  
ii. However, I found that the data given is up tp "2021-03" based on both the createdDate in User table, and the scannedDate in Receipt table.
Therefore, based on the given data:  

How do I define "past 6 month": starting for the most recent month in the dataset is the most recent month, then trace backward 6 months and include the most recent month.

And the correcsponding query is shown as stored in `query6`

##### Findings: "HY-VEE" is the brand with the most transactions among users who were created within the past 6 months.

In [33]:
query6 = """
    WITH maxMonth AS (
        SELECT MAX(month) AS max_month
        FROM (
            (SELECT dateScanned AS month
            FROM Receipt
            ORDER BY dateScanned DESC
            LIMIT 1)
            UNION ALL
            (SELECT createdDate AS month
            FROM User
            ORDER BY createdDate DESC
            LIMIT 1)
            ) AS combined_results
        ), 
        receiptsOfRecentUsers AS (
        Select r._id, r.totalSpent, u.createdDate
        FROM Receipt AS r LEFT JOIN User as u ON r.userId = u._id
        WHERE DATE_FORMAT(u.createdDate, '%Y-%m') >= (SELECT DATE_FORMAT(DATE_SUB(max_month, INTERVAL 6 MONTH), '%Y-%m') FROM maxMonth)
        )

    SELECT item.brandCode, COUNT(*) as Total
        FROM receiptsOfRecentUsers AS r LEFT JOIN ReceiptRewardItem AS item
            ON r._id = item.receiptId
        WHERE item.brandCode <> 'NULL'
        GROUP BY item.brandCode
        ORDER BY Total DESC
        LIMIT 1
    """
query_to_df(conn=engine, query = query6)

,brandCode,Total
0,HY-VEE,291


## 3. Data Quality Issues Evaluation

Summary:

a. There are Users who submitted receipts but do not have records in the User table  

b. Duplicate Users exist in the User table  

c. There are a lot of receipt reward items that don't have a brandCode or a field that could be mapped to an existing Brand record.  
This creates challenges to retrieve analysis summary related to brands  

d. There's no id assigned for each reward items  

e. Overlapping Brand categories: such as "Dairy" and "Dairy and Refrigerated"

f. In the receipt table, the values under column, `rewardsReceiptStatus` are confusing.
Currently, rewardsReceiptStatus values are 'FINISHED', 'REJECTED', 'FLAGGED', 'SUBMITTED', 'PENDING'
Based on questions on part 3, with assumption that these status should be exlusive of each other, I assume that 'FINISHED' means
a status that a receipt which has been scanned but it was not rejected, not flagged, not pending, and therefore, I inferred this 
staus to be equivalent as "ACCEPTED". However, this naming 'FINISHED' is confusing and if what I inferred is correct, this status
should be changed to 'ACCEPTED'.

[code used to find above quality issues are listed below]

In [34]:
# 3a. There are Users who submitted receipts but do not have records in the User table

hset = set(usersData._id)
unknownUserIds = []
for idx, id in enumerate(receiptsData.userId):
    if id not in hset:
        unknownUserIds.append(id)
print("{} user ids in Receipt table that doesn't have matched user ids in User tables\n".format(len(unknownUserIds)))


148 user ids in Receipt table that doesn't have matched user ids in User tables



In [35]:
# 3b. Duplicate Users exist in the User table  
# using code below we see that there are 70 user ids that has duplicate in the User table
num_dup_uids = sum(dataDict["users"]["_id.$oid"].value_counts() > 1)
print("{} user user ids that has duplicate in the User table\n".format(num_dup_uids))


70 user user ids that has duplicate in the User table



In [36]:
# 3c. There are a lot of receipt reward items that don't have a brandCode or a field that could be mapped to an existing Brand record
# makes it hard to perform analysis related to brands
missing_brandcode_percentage = sum(rewardsReceiptItems.brandCode.isna())/rewardsReceiptItems.shape[0] * 100
missing_barcode_percentage = sum(rewardsReceiptItems.barcode.isna())/rewardsReceiptItems.shape[0] * 100

print("{} % of reward items don't have a brandCode\n{} % of reward items don't have a barcode".format(round(missing_brandcode_percentage, 2), round(missing_barcode_percentage, 2)))


62.54 % of reward items don't have a brandCode
55.48 % of reward items don't have a barcode


In [37]:
# 3d. There's no id assigned for each reward items  
# Refer back to part 1, where the reward items are extracted from the Receipt table's column ``

rewardsReceiptItemsLst = []
num_nan = 0


def normalize_rewardsReceiptItems(row, dest = rewardsReceiptItemsLst):
    """
    Extract 'reward receipt item' from column `rewardsReceiptItemList`
    and map receipt id to each item.

    Args:
        row(Pandas series): a pandas series sliced from a dataframe as a row
        dest(list): a list where each processed item will be appended to
    
    Returns:
        None
    """
    global num_nan
    receipt_id = row["_id"]
    rewardsReceiptItemList = row["rewardsReceiptItemList"]
    if pd.isna(rewardsReceiptItemList).all() if isinstance(rewardsReceiptItemList, (list, pd.Series)) else pd.isna(rewardsReceiptItemList):
        num_nan += 1
        return 
    new_row = pd.json_normalize(rewardsReceiptItemList)
    new_row["receiptId"] = receipt_id
    dest.append(new_row)


# Load raw processed data from dataDict
receiptsData = dataDict["receipts"]

# Clean receipt table columns
receiptsDataRenameCols = {col: cleanColName(col) for col in receiptsData.columns if "." in col}
receiptsData = receiptsData.rename(columns=receiptsDataRenameCols)

# Convert all date columns to date object
for col in ['createDate', 'dateScanned',
        'finishedDate', 'modifyDate', 
        'pointsAwardedDate', 'purchaseDate']:
    receiptsData[col] = pd.to_datetime(receiptsData[col], unit="ms")

receiptsData.apply(normalize_rewardsReceiptItems, axis = 1)
rewardsReceiptItems = pd.concat(rewardsReceiptItemsLst, ignore_index= True)

actual_nan_rri = receiptsData["_id"].count() - receiptsData.rewardsReceiptItemList.count()

report1 = """
During extracting receipt reward items, found {} receipts that has missing value in column 'receiptsRewordsItemList'\n
Matched with actual missing values in the column: {}\n
Total number of receipt rewards items extracted: {}\n
"""
print(report1.format(num_nan, actual_nan_rri == num_nan, rewardsReceiptItems.shape[0] ))


During extracting receipt reward items, found 440 receipts that has missing value in column 'receiptsRewordsItemList'

Matched with actual missing values in the column: True

Total number of receipt rewards items extracted: 6941




In [38]:
cols_contains_id = [col for col in rewardsReceiptItems.columns if "id" in col.lower() or "brand" in col.lower()]
cols_contains_id

['partnerItemId',
 'receiptId',
 'pointsPayerId',
 'rewardsProductPartnerId',
 'brandCode',
 'metabriteCampaignId']

In [39]:
print("Among reward receipt items: \n")
matches = []
matchesDict = {}
for potential_match_col in cols_contains_id:
    for id in rewardsReceiptItems[potential_match_col]:
        if id in brandsData._id:
            matches.append(idx)
    if len(matches) > 1:
        matchesDict[potential_match_col] = matches
if len(matchesDict) > 1:
    for key, lst in matchesDict.items():
        print("In {} colume found {} matched brand id\n".format(key, len(lst)))
else:
    print("None of the columns has value matched with brand id")

brandCodeSet = set(brandsData.brandCode)
matchedBrands = set()
matchedBrandIndices = []

for idx,code in enumerate(rewardsReceiptItems['brandCode']):
    if code in brandCodeSet:
        matchedBrands.add(code)
        matchedBrandIndices.append(idx)
print("In {} colume, among {} unique values, found {} matched brand codes\n".format('brandCode', len(brandCodeSet), len(matchedBrands)))

barcodeSet = set(brandsData.barcode)
matchedBarcode = set()
matchedBarcodeIndices = []
for idx,code in enumerate(rewardsReceiptItems['barcode']):
    if code in barcodeSet:
        matchedBarcode.add(code)
        matchedBarcodeIndices.append(idx)
print("In {} colume, among {} unique values, found {} matched barcodes\n".format('barCode', len(barcodeSet), len(matchedBarcode)))

Among reward receipt items: 

None of the columns has value matched with brand id
In brandCode colume, among 898 unique values, found 42 matched brand codes

In barCode colume, among 1160 unique values, found 16 matched barcodes



In [40]:
len([i for i in range(rewardsReceiptItems.shape[0]) if i not in matchedBrandIndices and i in matchedBarcodeIndices])
# This means that there's no item that doesn't have a matched brand code but has a matched barcode

0

In [41]:
#3e.  Overlapping Brand categories in Brand table
brandsData.category.unique()

array(['Baking', 'Beverages', 'Candy & Sweets', 'Condiments & Sauces',
       'Canned Goods & Soups', nan, 'Magazines', 'Breakfast & Cereal',
       'Beer Wine Spirits', 'Health & Wellness', 'Beauty', 'Baby',
       'Frozen', 'Grocery', 'Snacks', 'Household', 'Personal Care',
       'Dairy', 'Cleaning & Home Improvement', 'Deli',
       'Beauty & Personal Care', 'Bread & Bakery', 'Outdoor',
       'Dairy & Refrigerated'], dtype=object)

In [42]:
# 3f. In the receipt table, values used in column `rewardsReceiptStatus` are confusing.
receiptsData.rewardsReceiptStatus.unique()

array(['FINISHED', 'REJECTED', 'FLAGGED', 'SUBMITTED', 'PENDING'],
      dtype=object)

## 4. Communicate with Stakeholders

Hi, dear product manager/ business analytics manager,

After reviewing the three data tables you shared with me, I have a few questions and observations regarding the data quality and analysis process:

- User Data Consistency:
    - Question: Are guest users allowed to submit scanned receipts, or is the User table an outdated version that could possibly not include all users?
    - Observation: I found user IDs in the Receipt table that do not exist in the User table, which raises concerns about data consistency.  

- Brand Code Accuracy:
    - Observation: Many reward items listed in receipts do not have a brand code, which can bias our results when identifying the "top k" brands.
    - Question: 
        - Do we have a list of reward items that I could store them to the database for easier lookup?
        - Do we have strategies or plans to improve the accuracy of brand labeling for future analyses? Are there new policies to handle unlabeled or mislabeled items?

- Brand Categories Issues:
    - Observation: found categories overlapped: "Dairy" and "Dairy and Refrigerated"
    - Question: may I have your approval to group "Dairy" brands into "Dairy and Refrigerated"?

- Frequency of Analysis:
    - Question: How often would you like me to run and report this analysis?
    - Suggestion: If the analysis is reported monthly, we could cache the data for the past 6 months to avoid repeated data loading. For weekly reports, creating a receipt-item table could reduce the need for complex joins. 

- Key Performance Metrics:

    - Question: What are the key performance metrics we use to measure the success of our product?


Thank you for your attention to these matters. I look forward to your insights and guidance.


